In [1]:
import os
import numpy as np
from skimage.feature import hog
#from skimage.io import imread
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
import joblib

In [4]:
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

pos_im_path = 'C:\\Users\\admin\\JupyterNotebook_works\\mtech-programing-labs\\S2\\Computer-vision\\Project\\image-dataset\\1'
neg_im_path = 'C:\\Users\\admin\\JupyterNotebook_works\\mtech-programing-labs\\S2\\Computer-vision\\Project\\image-dataset\\0'

In [5]:
pos_im_listing = os.listdir(pos_im_path)
neg_im_listing = os.listdir(neg_im_path)

print('No. images having Human: ', len(pos_im_listing), '\nNo. images not having Human: ', len(neg_im_listing))

No. images having Human:  559 
No. images not having Human:  362


In [9]:
data = []
labels = []

In [10]:
for file in pos_im_listing: 
    img = Image.open(pos_im_path + '\\' + file)
    img = img.resize((64,128))
    gray = img.convert('L') 
    hog_feat = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True)
    data.append(hog_feat)
    labels.append(1)

In [11]:
for file in neg_im_listing:
    img = Image.open(neg_im_path + '\\' + file)
    img = img.resize((64,128))
    gray = img.convert('L')
    hog_feat = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True) 
    data.append(hog_feat)
    labels.append(0)

In [12]:
data = np.array(data)

In [35]:
print(data.shape)

(921, 3780)


In [17]:
labels = np.array(labels)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=20, random_state=40, stratify=labels)

In [28]:
print((len(y_train[y_train==0])/y_train.shape[0])*100, 
      (len(y_train[y_train==1])/y_train.shape[0])*100, '\n',
      (len(y_test[y_test==0])/y_test.shape[0])*100,
      (len(y_test[y_test==1])/y_test.shape[0])*100)

39.289678135405104 60.710321864594896 
 40.0 60.0


In [29]:
model = LinearSVC()
model.fit(X_train, y_train)

C:\Users\admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

In [30]:
model.score(X_train, y_train)

0.9988901220865705

In [57]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88         8
           1       0.92      0.92      0.92        12

    accuracy                           0.90        20
   macro avg       0.90      0.90      0.90        20
weighted avg       0.90      0.90      0.90        20



In [33]:
confusion_matrix(y_test, y_pred)

array([[ 7,  1],
       [ 1, 11]], dtype=int64)

In [43]:
from sklearn.model_selection import GridSearchCV

In [47]:
param_grid = {'tol': [0.0001, 0.001, 0.00001],
              'C': [0.01, 0.1, 1.0, 10, 100],
              'fit_intercept': [True, False],} 
# penalty':['l2', 'l1'],
#             'loss': ['hinge', 'squared_hinge'],
              

In [48]:
grid = GridSearchCV(LinearSVC(), param_grid, refit=True, verbose=1, scoring='f1', cv=3)

In [49]:
grid.fit(data, labels)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear 

GridSearchCV(cv=3, estimator=LinearSVC(),
             param_grid={'C': [0.01, 0.1, 1.0, 10, 100],
                         'fit_intercept': [True, False],
                         'tol': [0.0001, 0.001, 1e-05]},
             scoring='f1', verbose=1)

In [52]:
grid.best_params_

{'C': 0.01, 'fit_intercept': True, 'tol': 0.0001}

In [53]:
grid.best_estimator_

LinearSVC(C=0.01)

In [54]:
clf_new = LinearSVC(C=0.01)
clf_new.fit(X_train, y_train)

LinearSVC(C=0.01)

In [55]:
clf_new.score(X_train, y_train)

0.93007769145394

In [58]:
y_pred_new = clf_new.predict(X_test)
print(classification_report(y_test, y_pred_new))

              precision    recall  f1-score   support

           0       0.86      0.75      0.80         8
           1       0.85      0.92      0.88        12

    accuracy                           0.85        20
   macro avg       0.85      0.83      0.84        20
weighted avg       0.85      0.85      0.85        20



In [59]:
confusion_matrix(y_test, y_pred_new)

array([[ 6,  2],
       [ 1, 11]], dtype=int64)

In [36]:
joblib.dump(model, 'svm_model.npy')

['svm_model.npy']